# PÖFF's movie info

In [4]:
import pandas as pd
import os
from datetime import date
from os import listdir
from os.path import isfile, join
import requests
import re
import imdb

import qgrid
#jupyter nbextension enable --py --sys-prefix qgrid
#jupyter nbextension enable --py --sys-prefix widgetsnbextension

def createP6ffMovieDataFile():

    p6ff_url = "https://poff.ee/en/search_film/"

    # Get movie titles
    movies_html = requests.get(p6ff_url)
    year_html = iter(movies_html.text.split("\n"))
    movies = []
    for i, line in enumerate(year_html):
        # find line wih movie titels
        if (line.find('<div class="card_film_title">') != -1):

            movie_titles = re.findall('<h3 class="film_title_locale dark_1" id="title">([a-zA-Z0-9 \-.:/]+)</h3>',line)

            #print (sorted(movie_titles))

    # Get movie data
    i = imdb.IMDb()
    movies = {}
    movies_data = {"title": [],
                  "rating": [],
                  "votes": [],
                  "year": [],
                  "genres": [],
                  "cover url": [],
                  "imdbID": [],
                  "imdbLink": [],
                  "imdb": []}
    for j,title in enumerate(sorted(movie_titles)):                    
        print ("\t",title)
        movies[title] = {}
        best_match = i.search_movie(title)

        if (j == 10):
            break
        movies_data["title"].append(title)


        #if no matches
        if len(best_match) == 0: 
            movies_data["rating"].append(0)
            movies_data["votes"].append(0)
            movies_data["year"].append(0)
            movies_data["genres"].append("")
            movies_data["cover url"].append("")
            movies_data["imdbID"].append("")
            movies_data["imdbLink"].append("")
            movies_data["imdb"].append("") 
            continue

        best_match =i.search_movie(title)[0] #first match is chosen
        i.update(best_match)
        movies[title]["imdb"] = best_match
        movies_data["imdb"].append(best_match)

        if "rating" in best_match.keys():
            movies_data["rating"].append(best_match["rating"])
        else:
            movies_data["rating"].append("");

        if "year" in best_match.keys():
            movies_data["year"].append(best_match["year"])
        else:
            movies_data["year"].append("");

        if "genres" in best_match.keys():
            movies_data["genres"].append(";".join(best_match["genres"]))
        else:
            movies_data["genres"].append("");

        if "votes" in best_match.keys():
            movies_data["votes"].append(best_match["votes"])
        else:
            movies_data["votes"].append("");

        if "cover url" in best_match.keys():
            movies_data["cover url"].append(best_match["cover url"])
        else:
            movies_data["cover url"].append("");

        if "imdbID" in best_match.keys():
            movies_data["imdbID"].append(best_match["imdbID"])
            movies_data["imdbLink"].append('<a href="http://www.imdb.com/title/tt' + best_match["imdbID"] + '/">Link</a>')
        else:
            movies_data["imdbID"].append("");
            movies_data["imdbLink"].append("");

    #print(" - ".join([title,str(movies[title]["rating"]),str(movies[title]["votes"]), str(movies[title]["year"]), ";".join(movies[title]["genres"])]))    

    # write data to file
    df = pd.DataFrame(movies_data, columns= ['title', 'rating', 'votes', 'year', 'genres' , 'imdbLink'])

    today = date.today()
    fileName = os.path.join("data","movies_data.csv")
    newFileName = os.path.join("data","movies_data_" + str(today) + ".csv")
    df.to_csv(newFileName, index = False, header=True)

# get all data files
onlyfiles = [f for f in listdir("data") if isfile(join("data", f))]

# if data files are missing create one
if (len(onlyfiles) == 0):
    createP6ffMovieDataFile()

# last imdb data file
lastFile = onlyfiles[len(onlyfiles)-1]

# extract data
df = pd.read_csv(os.path.join("data",lastFile)) 

# display data
df = df.sort_values("rating", ascending = False)
df
#qgrid_widget = qgrid.show_grid(df)
#qgrid_widget

,title,rating,votes,year,genres,imdbLink
2,50 or Two Whales Meet on the Beach,7.9,9,2020,Drama,"<a href=""http://www.imdb.com/title/tt12211524/..."
4,A Man Who Never Was,7.4,4440,1956,War;Drama,"<a href=""http://www.imdb.com/title/tt0049471/""..."
8,An Ordinary Woman 2,7.3,306,2018,Comedy;Drama,"<a href=""http://www.imdb.com/title/tt8561620/""..."
7,Ammonite,6.6,630,2020,Biography;Drama;Romance,"<a href=""http://www.imdb.com/title/tt7983894/""..."
5,A Yellow Animal,6.1,106,2020,Drama,"<a href=""http://www.imdb.com/title/tt7811018/""..."
9,And Tomorrow the Entire World,6.0,242,2020,Crime;Drama;Romance,"<a href=""http://www.imdb.com/title/tt7716120/""..."
6,Abulele,5.9,251,2015,Animation;Adventure;Family,"<a href=""http://www.imdb.com/title/tt4937208/""..."
3,A Distant Place,5.7,37,2010,Drama;Romance,"<a href=""http://www.imdb.com/title/tt1267494/""..."
0,101 Estonian Films: Animation,0.0,0,0,NaN,NaN
1,25 Years of Innocence. The Case of Tomek Komenda,0.0,0,0,NaN,NaN


In [5]:
# if last data files is not from today it will creat new one

today = date.today()
lastFileDate = lastFile.split(".")[0].split("_")[2]
if (str(today) != lastFileDate):
    print(lastFileDate)
    print(str(today))
    createP6ffMovieDataFile()

2020-11-16
2020-11-17
